In [678]:
import numpy as np
import matplotlib.pyplot as plt

In [679]:
from loaders import clean_text_for_pattern_letter
from temp_text import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
500
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [680]:
def predict_train(input_str, letters, max_index:int = 4,):
    next_index: dict[str, int] = {}
    index_by_prev: dict[str, dict] = {}

    StatsDict: dict[str, dict] = {}

    for letter in letters:
        next_index[letter] = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]
        
        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict[stats_key].setdefault(next_inp, 0)

        StatsDict[stats_key][next_inp] += 1

        by_prev_key = prev_inp + str(prev_index) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1

        by_prev_key = prev_inp + str(0) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1
        
        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index:
            next_index[prev_inp] = 1
    
    return StatsDict, index_by_prev

In [681]:
def predict(input_str, TF:list, StatsDict:dict, index_by_prev:dict, max_index, err = 0,):
    prev_index_l = [x for x in range(1, max_index+1)]

    output = []

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            # B B {'B': 1} [3] {3: 1} [3] False
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [682]:
# train
max_index_L1 = 4

index_by_prev_L1: dict[str, dict] = {}
StatsDict_L1: dict[str, dict] = {}

StatsDict, index_by_prev_L1 = predict_train(letters=letters, input_str=input_str, max_index=max_index_L1)

In [683]:
for sdk, sdv in index_by_prev_L1.items():
    print(sdk, sdv)
    print('')

t1:h {1: 2}

t0:h {1: 4, 2: 2, 3: 3, 4: 2}

h1:e {1: 1, 4: 3}

h0:e {1: 1, 3: 3, 4: 5, 2: 4}

e1:  {1: 2, 4: 1}

e0:  {1: 6, 3: 4, 4: 4, 2: 3}

 1:s {1: 1}

 0:s {1: 1, 3: 2, 4: 1, 2: 2}

s1:k {1: 1}

s0:k {1: 1}

k1:y {1: 1}

k0:y {1: 1}

y1:  {2: 1}

y0:  {2: 1, 1: 2, 3: 1}

 2:a {1: 1}

 0:a {1: 3, 3: 5, 2: 2, 4: 2}

a1:b {1: 1}

a0:b {1: 1}

b1:o {1: 1}

b0:o {1: 1, 2: 2}

o1:v {1: 1}

o0:v {1: 1}

v1:e {2: 1}

v0:e {2: 1, 3: 2, 1: 1}

e2:  {3: 3, 2: 1, 1: 2}

 3:t {2: 2, 3: 1, 4: 1}

 0:t {2: 5, 4: 3, 1: 3, 3: 3}

t2:h {2: 2, 1: 2, 4: 1}

h2:e {3: 2, 2: 2}

e3:  {4: 2, 1: 1, 2: 2}

 4:p {1: 1}

 0:p {1: 2, 4: 1}

p1:o {2: 1}

p0:o {2: 1}

o2:r {1: 2}

o0:r {1: 3, 2: 2, 4: 1}

r1:t {3: 1}

r0:t {3: 1}

t3:  {1: 1}

t0:  {1: 2, 4: 1}

 1:w {1: 1}

 0:w {1: 1, 2: 2, 4: 2, 3: 2}

w1:a {2: 1}

w0:a {2: 3, 1: 2}

a2:s {2: 2, 1: 1}

a0:s {2: 2, 1: 3, 3: 3}

s2:  {2: 1, 4: 1}

s0:  {2: 2, 4: 7, 1: 2, 3: 3}

 2:t {4: 1, 2: 1, 3: 2}

t4:h {3: 2, 4: 1}

h3:e {4: 2, 2: 1}

e4:  {3: 1, 1: 1, 4

In [684]:
for sdk, sdv in StatsDict.items():
    print(sdk, sdv)
    print('')

t1:h1 {'e': 2}

h1:e1 {' ': 1}

e1: 1 {'s': 1, 'd': 1}

 1:s1 {'k': 1}

s1:k1 {'y': 1}

k1:y1 {' ': 1}

y1: 2 {'a': 1}

 2:a1 {'b': 1}

a1:b1 {'o': 1}

b1:o1 {'v': 1}

o1:v1 {'e': 1}

v1:e2 {' ': 1}

e2: 3 {'t': 1, 's': 1, 'c': 1}

 3:t2 {'h': 2}

t2:h2 {'e': 2}

h2:e3 {' ': 2}

e3: 4 {'p': 1, 'f': 1}

 4:p1 {'o': 1}

p1:o2 {'r': 1}

o2:r1 {'t': 1, 'd': 1}

r1:t3 {' ': 1}

t3: 1 {'w': 1}

 1:w1 {'a': 1}

w1:a2 {'s': 1}

a2:s2 {' ': 1, 'e': 1}

s2: 2 {'t': 1}

 2:t4 {'h': 1}

t4:h3 {'e': 1, 'i': 1}

h3:e4 {' ': 1, 'a': 1}

e4: 3 {'c': 1}

 3:c1 {'o': 1}

c1:o3 {'l': 1}

o3:l1 {'o': 1}

l1:o4 {'r': 1}

o4:r2 {' ': 2}

r2: 4 {'o': 1}

 4:o1 {'f': 1}

o1:f1 {' ': 1}

f1: 1 {'t': 1}

 1:t1 {'e': 1}

t1:e1 {'l': 1, 's': 1}

e1:l2 {'e': 1}

l2:e2 {'v': 1}

e2:v2 {'i': 1, 'e': 1}

v2:i1 {'s': 1}

i1:s3 {'i': 1}

s3:i2 {'o': 1}

i2:o2 {'n': 1}

o2:n1 {',': 1, 'o': 1}

n1:,1 {' ': 1}

,1: 2 {'t': 1}

 2:t2 {'u': 1}

t2:u1 {'n': 1}

u1:n2 {'e': 1}

n2:e3 {'d': 1}

e3:d1 {' ': 1}

d1: 3 {'t': 1}



In [685]:
# predict
TF = []

TF, err, out_L1 = predict(input_str=input_str, TF=TF, StatsDict=StatsDict, index_by_prev=index_by_prev_L1, max_index=max_index_L1)

print(err, err / (len(input_str) / 100) )

e e {'e': 4} [1, 2, 4, 3] {1: 4, 2: 2, 4: 2, 3: 3} [1, 2, 3, 4] False
    {' ': 4, 'a': 1, 't': 1, 'r': 1} [1, 4, 3, 2] {1: 1, 4: 5, 3: 3, 2: 4} [1, 2, 4, 3] False
s s {'s': 2, 'd': 2, 'm': 1, 'c': 1} [1, 4, 3, 2] {1: 6, 4: 4, 3: 4, 2: 3} [1, 4, 3, 2] False
p k {'k': 1, 'a': 1, 'o': 1, 'p': 2, 'h': 1} [1, 4, 3, 2] {1: 1, 4: 1, 3: 2, 2: 2} [1, 4, 3, 2] False
y y {'y': 1} [1] {1: 1} [1, 4, 3, 2] False
    {' ': 1} [1] {1: 1} [1] False
a a {'a': 1} [2] {2: 1} [1] False
b b {'b': 1} [1] {1: 1} [2] False
o o {'o': 1} [1] {1: 1} [1] False
v v {'v': 1} [1] {1: 1} [1] False
e e {'e': 1} [1] {1: 1} [1] False
    {' ': 1} [2] {2: 1} [1] False
c t {'t': 1, 's': 1, 'c': 2, 'i': 1, 'f': 1} [3, 2, 1] {3: 3, 2: 1, 1: 2} [2] False
h h {'h': 2, 'o': 1, 'e': 1} [2, 3, 4, 1] {2: 4, 3: 3, 4: 3, 1: 1} [3, 2, 1] False
e e {'e': 4, 'r': 1} [2, 1, 4, 3] {2: 2, 1: 4, 4: 2, 3: 3} [2, 3, 4, 1] False
    {' ': 4, 'r': 1, 'a': 1, 't': 1} [3, 2, 1, 4] {3: 3, 2: 4, 1: 1, 4: 5} [2, 1, 4, 3] False
s p {'p': 1, 'f': 1,

In [686]:
TF = np.array(TF, dtype=float)
# print(TF)
TF = np.pad(TF, (1, 1), constant_values=[1,1])
# print(TF)
TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (1, 0))[:-1] 
# TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (0, 1))[-1:]
# print(TF)
TF[np.where(TF < 0)] = 0
# TF = np.abs(TF)
TF[np.where(TF <= np.mean(TF[np.where(TF > 0)]))] = 0
# # TF[np.where(TF <= 0.5)] = 0
# print(TF.size, TF.max(), TF.min())

# fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
# # axs.plot(np.arange(TF.size), TF)
# markerline, stemline, baseline, = axs.stem(np.arange(TF.size), TF)
# plt.setp(markerline, markersize = 1)
# axs.xaxis.set_ticks(np.arange(len(input_str)) ,input_str)
# # axs.xaxis.grid(True)

# plt.show()

In [687]:
upstream_inp: dict[int, list] = {}

temp_seq = []

for inp_idx, inp in enumerate(input_str):
    tf = TF[inp_idx]
    
    if tf > 0:
        upstream_inp.setdefault(inp_idx, temp_seq.copy())
        temp_seq.clear()

    temp_seq.append(inp)

for item, val in upstream_inp.items():
    print(item, val)
    
print(len(upstream_inp))

5 ['t', 'h', 'e', ' ', 's']
18 ['k', 'y', ' ', 'a', 'b', 'o', 'v', 'e', ' ', 't', 'h', 'e', ' ']
32 ['p', 'o', 'r', 't', ' ', 'w', 'a', 's', ' ', 't', 'h', 'e', ' ', 'c']
37 ['o', 'l', 'o', 'r', ' ']
53 ['o', 'f', ' ', 't', 'e', 'l', 'e', 'v', 'i', 's', 'i', 'o', 'n', ',', ' ', 't']
59 ['u', 'n', 'e', 'd', ' ', 't']
66 ['o', ' ', 'a', ' ', 'd', 'e', 'a']
72 ['d', ' ', 'c', 'h', 'a', 'n']
116 ['n', 'e', 'l', '.', ' ', '"', 'i', 't', "'", 's', ' ', 'n', 'o', 't', ' ', 'l', 'i', 'k', 'e', ' ', 'i', "'", 'm', ' ', 'u', 's', 'i', 'n', 'g', ',', '"', ' ', 'c', 'a', 's', 'e', ' ', 'h', 'e', 'a', 'r', 'd', ' ', 's']
124 ['o', 'm', 'e', 'o', 'n', 'e', ' ', 's']
135 ['a', 'y', ',', ' ', 'a', 's', ' ', 'h', 'e', ' ', 's']
155 ['h', 'o', 'u', 'l', 'd', 'e', 'r', 'e', 'd', ' ', 'h', 'i', 's', ' ', 'w', 'a', 'y', ' ', 't', 'h']
166 ['r', 'o', 'u', 'g', 'h', ' ', 't', 'h', 'e', ' ', 'c']
175 ['r', 'o', 'w', 'd', ' ', 'a', 'r', 'o', 'u']
183 ['n', 'd', ' ', 't', 'h', 'e', ' ', 'd']
187 ['o', 'o', 'r',

In [688]:
letters_dict: dict[str, int] = {}
for let_idx, letter in enumerate(letters):
    letters_dict.setdefault(letter, let_idx)

upstream_patterns = []

for upstream_letters in upstream_inp.values():
    upstream_pattern = np.zeros(len(letters))

    for letter in upstream_letters:
        pattern_idx = letters_dict.get(letter, 0)

        upstream_pattern[pattern_idx] = 1
    
    upstream_patterns.append(upstream_pattern)

upstream_patterns = np.array(upstream_patterns)

for up_ptr in upstream_patterns:
    print(up_ptr.astype(int).tolist())

[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 1, 0, 0,

In [689]:
layer_size = 16

I = np.zeros(upstream_patterns.shape[1])
O = np.zeros(layer_size)
W = np.ones((O.size, I.size))
print('W', W.shape, W.max(), W.min(), np.median(W))
WinC = np.ones(O.shape)

k = 1
push_delta = 0.4

prec = 1e-30

epoch_cnt = 100
eps0 = 0.02

AVG_O = np.zeros(O.shape)
AVG_O_CNT = np.zeros(O.shape)

samples_idx = np.arange(upstream_patterns.shape[0])

for epoch_idx in range(epoch_cnt):
    eps = (epoch_cnt - epoch_idx) * eps0

    np.random.shuffle(samples_idx)

    for i_idx in samples_idx:
        I = upstream_patterns[i_idx]

        # infer
        O = np.dot(W/WinC[np.newaxis].T, I)
        # orig_O = np.copy(O)

        # # threshold
        # O[O < AVG_O] = np.min(O)

        # # avg O
        # if np.max(AVG_O_CNT) <= 0:
        #     AVG_O = np.copy(orig_O)
        # else:
        #     AVG_O = AVG_O + ((orig_O - AVG_O) / AVG_O_CNT)
        # AVG_O_CNT += 1

        # train
        # is_tr = False
        # if np.max(O) <= np.min(O):
        #     O = orig_O
        #     is_tr = True

        winner_ids = np.argsort(O)[::-1][:1+k]
        winner = winner_ids[0]
        loosers = winner_ids[1:1+k]

        # if is_tr:
        #     AVG_O[winner_ids] = O[winner_ids]
        
        dW = np.zeros(W.shape)
        dW[winner] = I * O[winner]
        dW[loosers] = -1 * I * push_delta * O[loosers] 

        WinC[winner] += 1

        # norm 
        nc = np.linalg.norm(dW, ord=1)
        if nc < prec:
            nc = prec
        dW = np.true_divide(dW, nc)

        # update
        W += dW

W = W/WinC[np.newaxis].T
print('W', W.shape, W.max(), W.min(), np.median(W))

W (16, 32) 1.0 1.0 1.0
W (16, 32) 1.0 -0.6736903104422843 0.008475972596999226


In [690]:
SL_symbols = [x.capitalize() for x in letters][:layer_size]
print(len(SL_symbols))

SLSymbolDict = {}

SL_symbols_d = {}
for idx, val in enumerate(SL_symbols):
    SL_symbols_d.setdefault(val + ':' + str(idx), 0)

upstream_inp_keys = list(upstream_inp.keys())

upstream_cluster_by_symbol: dict[str, int] = {}
for idx, val in enumerate(SL_symbols):
    upstream_cluster_by_symbol.setdefault(val, idx)

for upstr_idx, up_ptr in enumerate(upstream_patterns):
    O = np.dot(W, up_ptr)

    winner = np.argmax(O)
    symbol = SL_symbols[winner]
    print(SL_symbols[winner])

    # print(W[upstream_cluster_by_symbol[symbol]].tolist())

    sls = SL_symbols[winner]
    sls_key = upstream_inp_keys[upstr_idx]
    SLSymbolDict.setdefault(sls_key, sls)
    
    SL_symbols_d[SL_symbols[winner] + ':' + str(winner)] += 1

print(len(SL_symbols_d))
for key, val in SL_symbols_d.items():
    print(key, val)

print(SLSymbolDict)

16
D
N
F
P
G
E
F
L
A
E
N
A
K
K
L
P
F
A
A
A
G
A
F
L
C
B
C
J
H
D
C
O
E
N
16
A:0 6
B:1 1
C:2 3
D:3 2
E:4 3
F:5 4
G:6 2
H:7 1
I:8 0
J:9 1
K:10 2
L:11 3
M:12 0
N:13 3
O:14 1
P:15 2
{5: 'D', 18: 'N', 32: 'F', 37: 'P', 53: 'G', 59: 'E', 66: 'F', 72: 'L', 116: 'A', 124: 'E', 135: 'N', 155: 'A', 166: 'K', 175: 'K', 183: 'L', 187: 'P', 198: 'F', 237: 'A', 266: 'A', 311: 'A', 355: 'G', 375: 'A', 384: 'F', 392: 'L', 404: 'C', 415: 'B', 429: 'C', 436: 'J', 455: 'H', 459: 'D', 465: 'C', 468: 'O', 484: 'E', 495: 'N'}


In [691]:
sl_input = list(SLSymbolDict.values())
print(sl_input)

['D', 'N', 'F', 'P', 'G', 'E', 'F', 'L', 'A', 'E', 'N', 'A', 'K', 'K', 'L', 'P', 'F', 'A', 'A', 'A', 'G', 'A', 'F', 'L', 'C', 'B', 'C', 'J', 'H', 'D', 'C', 'O', 'E', 'N']


In [692]:
# SL train
max_index_L2 = 4

StatsDict_L2, index_by_prev_L2 = predict_train(input_str=sl_input, letters=SL_symbols, max_index=max_index_L2)

In [693]:
for sdk, sdv in StatsDict_L2.items():
    print(sdk, sdv)
    print('')

D1:N1 {'F': 1}

N1:F1 {'P': 1}

F1:P1 {'G': 1}

P1:G1 {'E': 1}

G1:E1 {'F': 1}

E1:F2 {'L': 1}

F2:L1 {'A': 1}

L1:A1 {'E': 1}

A1:E2 {'N': 1}

E2:N2 {'A': 1}

N2:A2 {'K': 1}

A2:K1 {'K': 1}

K1:K1 {'L': 1}

K2:L2 {'P': 1}

L2:P2 {'F': 1}

P2:F3 {'A': 1}

F3:A3 {'A': 1}

A3:A3 {'A': 1}

A4:A4 {'G': 1}

A1:G2 {'A': 1}

G2:A2 {'F': 1}

A2:F4 {'L': 1}

F4:L3 {'C': 1}

L3:C1 {'B': 1}

C1:B1 {'C': 1}

B1:C2 {'J': 1}

C2:J1 {'H': 1}

J1:H1 {'D': 1}

H1:D2 {'C': 1}

D2:C3 {'O': 1}

C3:O1 {'E': 1}

O1:E3 {'N': 1}



In [694]:
for sdk, sdv in index_by_prev_L2.items():
    print(sdk, sdv)
    print('')

D1:N {1: 1}

D0:N {1: 1}

N1:F {1: 1}

N0:F {1: 1}

F1:P {1: 1}

F0:P {1: 1}

P1:G {1: 1}

P0:G {1: 1}

G1:E {1: 1}

G0:E {1: 1}

E1:F {2: 1}

E0:F {2: 1}

F2:L {1: 1}

F0:L {1: 1, 3: 1}

L1:A {1: 1}

L0:A {1: 1}

A1:E {2: 1}

A0:E {2: 1}

E2:N {2: 1}

E0:N {2: 1}

N2:A {2: 1}

N0:A {2: 1}

A2:K {1: 1}

A0:K {1: 1}

K1:K {1: 1}

K0:K {1: 1}

K2:L {2: 1}

K0:L {2: 1}

L2:P {2: 1}

L0:P {2: 1}

P2:F {3: 1}

P0:F {3: 1}

F3:A {3: 1}

F0:A {3: 1}

A3:A {3: 1}

A0:A {3: 1, 4: 1}

A4:A {4: 1}

A1:G {2: 1}

A0:G {2: 1}

G2:A {2: 1}

G0:A {2: 1}

A2:F {4: 1}

A0:F {4: 1}

F4:L {3: 1}

L3:C {1: 1}

L0:C {1: 1}

C1:B {1: 1}

C0:B {1: 1}

B1:C {2: 1}

B0:C {2: 1}

C2:J {1: 1}

C0:J {1: 1}

J1:H {1: 1}

J0:H {1: 1}

H1:D {2: 1}

H0:D {2: 1}

D2:C {3: 1}

D0:C {3: 1}

C3:O {1: 1}

C0:O {1: 1}

O1:E {3: 1}

O0:E {3: 1}



In [695]:
# predict
TF_L2 = []

TF_L2, err_L2, out_L2 = predict(input_str=sl_input, 
                                TF=TF_L2, 
                                StatsDict=StatsDict_L2, 
                                index_by_prev=index_by_prev_L2, 
                                max_index=max_index_L2)

for o in out_L2:
    print(o)

print(err_L2, err_L2 / (len(sl_input) / 100), len(sl_input), len(out_L2) )

F F {'F': 1} [1] {1: 1} [1, 2, 3, 4] False
P P {'P': 1} [1] {1: 1} [1] False
G G {'G': 1} [1] {1: 1} [1] False
E E {'E': 1} [1] {1: 1} [1] False
F F {'F': 1} [1] {1: 1} [1] False
L L {'L': 1} [2] {2: 1} [1] False
A A {'A': 1} [1] {1: 1} [2] False
E E {'E': 1} [1] {1: 1} [1] False
N N {'N': 1} [2] {2: 1} [1] False
A A {'A': 1} [2] {2: 1} [2] False
K K {'K': 1} [2] {2: 1} [2] False
K K {'K': 1} [1] {1: 1} [2] False
L L {'L': 1} [1] {2: 1} [1] True
P P {'P': 1} [2] {2: 1} [2] False
F F {'F': 1} [2] {2: 1} [2] False
A A {'A': 1} [3] {3: 1} [2] False
A A {'A': 1} [3] {3: 1} [3] False
A A {'A': 1} [3] {4: 1} [3] True
G G {'G': 1} [4] {1: 1} [4] True
A A {'A': 1} [2] {2: 1} [1] False
F F {'F': 1} [2] {2: 1} [2] False
L L {'L': 1} [4] {4: 1} [2] False
C C {'C': 1} [3] {3: 1} [4] False
B B {'B': 1} [1] {1: 1} [3] False
C C {'C': 1} [1] {1: 1} [1] False
J J {'J': 1} [2] {2: 1} [1] False
H H {'H': 1} [1] {1: 1} [2] False
D D {'D': 1} [1] {1: 1} [1] False
C C {'C': 1} [2] {2: 1} [1] False
O O {'O'

In [696]:
# from itertools import cycle

def ds_predict(input_str, 
               StatsDict: dict, 
               index_by_prev: dict,
               upstream_cluster_by_symbol: dict,
               letters_dict: dict,
               out_L2: list,
               SLSymbolDict: dict,
               W,
               max_index):
    prev_index_l = [x for x in range(1, max_index+1)]

    err = 0
    TF = []
    output = []

    ds_out_idx = 0
    ds_inp = '0'
    ds_pattern = np.ones(W.shape[1])

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        # downstream pattern
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
                ds_pattern = np.ones(W.shape[1])
            else:
                ds_inp = out_L2[ds_out_idx]
                ds_pattern = W[upstream_cluster_by_symbol.get(ds_inp)]
                # ds_pattern = np.copy(W[upstream_cluster_by_symbol.get(ds_inp)])
                # ds_pattern[np.where(ds_pattern < 0)] = 0

                ds_out_idx += 1
        print(stats_d)
        for st_key, st in stats_d.items():
            letter_idx = letters_dict.get(st_key)
            
            ds_stats = ds_pattern[letter_idx]
            print(st_key, ds_stats)
            if ds_stats <= 0:
                stats_d[st_key] = 0

                continue
            
            stats_d[st_key] *= ds_stats
        
        print(stats_d)
        print(input_str[inp_idx + 1])
        print()

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [697]:
dsp_TF, dsp_err, dsp_output = ds_predict(input_str=input_str,
                                         StatsDict=StatsDict,
                                         index_by_prev=index_by_prev_L1,
                                         upstream_cluster_by_symbol=upstream_cluster_by_symbol,
                                         letters_dict=letters_dict,
                                         out_L2=out_L2,
                                         SLSymbolDict=SLSymbolDict,
                                         W=W,
                                         max_index=max_index_L1)

print(dsp_err, dsp_err / (len(input_str) / 100))


{'e': 4}
e 1.0
{'e': 4.0}
e

{' ': 4, 'a': 1, 't': 1, 'r': 1}
  1.0
a 1.0
t 1.0
r 1.0
{' ': 4.0, 'a': 1.0, 't': 1.0, 'r': 1.0}
 

{'s': 2, 'd': 2, 'm': 1, 'c': 1}
s 1.0
d 1.0
m 1.0
c 1.0
{'s': 2.0, 'd': 2.0, 'm': 1.0, 'c': 1.0}
s

{'k': 1, 'a': 1, 'o': 1, 'p': 2, 'h': 1}
k 1.0
a 1.0
o 1.0
p 1.0
h 1.0
{'k': 1.0, 'a': 1.0, 'o': 1.0, 'p': 2.0, 'h': 1.0}
k

{'y': 1}
y 0.7326914556911247
{'y': 0.7326914556911247}
y

{' ': 1}
  0.5982991443412432
{' ': 0.5982991443412432}
 

{'a': 1}
a 0.6998540760405503
{'a': 0.6998540760405503}
a

{'b': 1}
b 0.21563739829605602
{'b': 0.21563739829605602}
b

{'o': 1}
o 0.14577911270958935
{'o': 0.14577911270958935}
o

{'v': 1}
v 0.21772602439440497
{'v': 0.21772602439440497}
v

{'e': 1}
e 0.5864788919526374
{'e': 0.5864788919526374}
e

{' ': 1}
  0.5982991443412432
{' ': 0.5982991443412432}
 

{'t': 1, 's': 1, 'c': 2, 'i': 1, 'f': 1}
t 0.06864127296099086
s 0.41023381316389285
c -0.06481739191798727
i -0.030674947765641773
f 0.1881438930550981
{'t': 0.06864

In [698]:
def downstream_predict_train(input_str: str | list,
                             sl_input: list,
                             SLSymbolDict: dict,
                            #  ds_out: list,
                             StatsDict_L1: dict,
                            #  StatsDict_L2,
                            #  TF_L1,
                             index_by_prev_L1: dict,
                            #  index_by_prev_L2,
                             max_index_L1: int,
                            #  max_index_L2,
                             ):
    next_index: dict[str, int] = {}

    for letter in letters:
        next_index[letter] = 1

    ds_inp = '0'
    sl_inp_idx = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]

        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if sl_inp_idx == len(sl_input):
                ds_inp = '0'
            else:
                ds_inp = sl_input[sl_inp_idx]
                sl_inp_idx += 1
        # print(f'{ds_inp=}')
        stats_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict_L1.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict_L1[stats_key].setdefault(next_inp, 0)

        StatsDict_L1[stats_key][next_inp] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(0) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index_L1:
            next_index[prev_inp] = 1
        
    return StatsDict_L1, index_by_prev_L1


In [699]:
StatsDict_L1_ds = {}
index_by_prev_L1_ds = {}

StatsDict_L1_ds, index_by_prev_L1_ds = downstream_predict_train(input_str=input_str,
                                                                sl_input=sl_input,
                                                                SLSymbolDict=SLSymbolDict,
                                                                StatsDict_L1=StatsDict_L1_ds,
                                                                index_by_prev_L1=index_by_prev_L1_ds,
                                                                max_index_L1=max_index_L1, )

for key, val in StatsDict_L1_ds.items():
    print(key, val)

0:t1:h1 {'e': 1}
0:h1:e1 {' ': 1}
0:e1: 1 {'s': 1}
0: 1:s1 {'k': 1}
N:s1:k1 {'y': 1}
N:k1:y1 {' ': 1}
N:y1: 2 {'a': 1}
N: 2:a1 {'b': 1}
N:a1:b1 {'o': 1}
N:b1:o1 {'v': 1}
N:o1:v1 {'e': 1}
N:v1:e2 {' ': 1}
N:e2: 3 {'t': 1}
N: 3:t2 {'h': 1}
N:t2:h2 {'e': 1}
N:h2:e3 {' ': 2}
N:e3: 4 {'p': 1}
F: 4:p1 {'o': 1}
F:p1:o2 {'r': 1}
F:o2:r1 {'t': 1}
F:r1:t3 {' ': 1}
F:t3: 1 {'w': 1}
F: 1:w1 {'a': 1}
F:w1:a2 {'s': 1}
F:a2:s2 {' ': 1}
F:s2: 2 {'t': 1}
F: 2:t4 {'h': 1}
F:t4:h3 {'e': 1}
F:h3:e4 {' ': 1}
F:e4: 3 {'c': 1}
F: 3:c1 {'o': 1}
P:c1:o3 {'l': 1}
P:o3:l1 {'o': 1}
P:l1:o4 {'r': 1}
P:o4:r2 {' ': 1}
P:r2: 4 {'o': 1}
G: 4:o1 {'f': 1}
G:o1:f1 {' ': 1}
G:f1: 1 {'t': 1}
G: 1:t1 {'e': 1}
G:t1:e1 {'l': 1, 's': 1}
G:e1:l2 {'e': 1}
G:l2:e2 {'v': 1}
G:e2:v2 {'i': 1}
G:v2:i1 {'s': 1}
G:i1:s3 {'i': 1}
G:s3:i2 {'o': 1}
G:i2:o2 {'n': 1}
G:o2:n1 {',': 1}
G:n1:,1 {' ': 1}
G:,1: 2 {'t': 1}
G: 2:t2 {'u': 1}
E:t2:u1 {'n': 1}
E:u1:n2 {'e': 1}
E:n2:e3 {'d': 1}
E:e3:d1 {' ': 1}
E:d1: 3 {'t': 1}
E: 3:t3 {'o': 1}
F:t3:o

In [700]:
from itertools import cycle

print(out_L2)
def downstream_predict(input_str: str | list,
                       SLSymbolDict: dict,
                       StatsDict_L1_ds: dict,
                       index_by_prev_L1_ds: dict,
                       out_L2: list,
                       max_index_L1: int,
                       TF: list,
                       err=0,):
    prev_index_l = [x for x in range(1, max_index_L1+1)]

    ds_inp = '0'
    ds_out_idx = 0
    ds_predict_out = cycle(out_L2)

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]

        # print(inp, SLSymbolDict.get(inp_idx, 0))
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
            else:
                ds_inp = out_L2[ds_out_idx]
                ds_out_idx += 1
        print(ds_inp, is_peak)
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev_L1_ds.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = ds_inp + ':' + prev_inp + \
                        str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict_L1_ds.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index_L1:
                max_key_d = 1

            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)

        prev_index_l = list(index_d.keys())

        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)

    return TF, err


['N', 'F', 'P', 'G', 'E', 'F', 'L', 'A', 'E', 'N', 'A', 'K', 'K', 'L', 'P', 'F', 'A', 'A', 'A', 'G', 'A', 'F', 'L', 'C', 'B', 'C', 'J', 'H', 'D', 'C', 'O', 'E', 'N']


In [701]:
TF_L1_ds = []

TF_L1_ds, err_L1_ds = downstream_predict(input_str=input_str,
                                         SLSymbolDict=SLSymbolDict,
                                         StatsDict_L1_ds=StatsDict_L1_ds,
                                         index_by_prev_L1_ds=index_by_prev_L1_ds,
                                         out_L2=out_L2,
                                         max_index_L1=max_index_L1,
                                         TF=TF_L1_ds, )

print(err_L1_ds, err_L1_ds / (len(input_str) / 100))

0 0
0 0
0 0
0 0
N D
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
F N
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
P F
P 0
P 0
P 0
P 0
G P
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
E G
E 0
E 0
E 0
E 0
E 0
F E
F 0
F 0
F 0
F 0
F 0
F 0
L F
L 0
L 0
L 0
L 0
L 0
A L
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
E A
E 0
E 0
E 0
E 0
E 0
E 0
E 0
N E
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
N 0
A N
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
K A
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K K
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
L K
L 0
L 0
L 0
L 0
L 0
L 0
L 0
P L
P 0
P 0
P 0
F P
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
A F
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A A
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
